In [91]:
# Install OpenAI library
!pip install -U -q openai tenacity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 6.9 MB/s eta 0:00:00


In [92]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
import os
os.chdir('/content/drive/MyDrive/GenAI_Course_Master/Course_1_ShopAssistAI/Week_1')
!ls


laptop_data.csv		restaurant_dataset_new.csv		    Shop_Assist_Chatbot.ipynb
OpenAI_API_Key.txt	restaurant_shopassist.ipynb		    updated_laptop.csv
restaurant_dataset.csv	ShopAssistAI_Demonstration_v2_Stage1.ipynb


In [94]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.width', 100)
df = pd.read_csv('restaurant_dataset_new.csv')
df.head()

,Restaurant Name,Cuisine Preferences,Dietary Restrictions,Location,User Reviews
0,Sri Sai Akshaya,South Indian,Vegan-friendly,Innespeta,Sri Sai Akshaya is the go-to place in Innespet...
1,Swagath Grand,North Indian,Gluten-free options,Morampudi,Swagath Grand is the go-to place in Morampudi ...
2,Ravi's Kitchen,Chinese,No restrictions,Aryapuram,I had a great time at Ravi's Kitchen in Aryapu...
3,Hotel River Bay,Continental,Vegan-friendly,Danavai Peta,Hotel River Bay in Danavai Peta is fantastic! ...
4,Masala Darbar,Italian,Vegetarian-friendly,Alcot Gardens,"Visited Masala Darbar at Alcot Gardens, and I ..."


In [95]:
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [96]:
openai.api_key = open('OpenAI_API_Key.txt', 'r').read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

# read the OpenAI API key


`Stage 1`

- Intent Clarity Layer
- Intent Confirmation Layer

`Stage 2`

- Restaurant Mapping Layer
- Restaurant sepcification Extraction Layer

`Stage 3`

- Restaurant Recommendation Layer

##### Major functions behind the Restaurant Shop Assist Chatbot

Let's now look at a brief overview of the major functions that form the chatbot.



- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's choice clearly. Specifically, this checks if the following properties for the user has been captured or not Cuisine preferences, Dietary restrictions, Location.
- `dictionary_present()`: This function checks if the final understanding of user's choice is returned by the chatbot as a python dictionary or not. If there is a dictionary, it extracts the information as a Python dictionary.
- `restaurant_specs_map_layer()`: This function maps restaurant specifications for each restaurant present in dataframe to a dictionary format.
- `compare_restaurant_specs_with_user`: This function compares the user's choice with different restaurant specifications and returns all matched restaurants.
- `initialize_conv_reco()`: Initializes the recommendations conversation i.e, recommends matched restaurants in short and readable format.
- `recommendation_validation()` : This function makes sure that matched restaurants are not null.


# Stage 1

In [97]:
# This function sets the system message(i.e, the behaviour of restaurant shopassist chatbot) to initialize conversation
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'Cuisine preferences': "South Indian",
                         'Dietary restrictions': "Vegan-friendly",
                         'Location': "Innespeta"}

    example_user_req = {'Cuisine preferences': "_",
                        'Dietary restrictions': "_",
                        'Location': "_"}
    example_user_req_Diet = {'Cuisine preferences': "_",
                        'Dietary restrictions': "Vegan-friendly",
                        'Location': "_"}
    example_user_req_diet_loc = {'Cuisine preferences': "_",
                        'Dietary restrictions': "Vegan-friendly",
                        'Location': "Innespeta"}

    Dietary_Restrictions = ["No restrictions", "Vegan-friendly", "Vegetarian-friendly", "Gluten-free options"]
    Cuisine_Preferences = ["South Indian", "North Indian", "Chinese", "Continental", "Italian", "Andhra Cuisine"]
    Restaurant_Locations = ["Katheru", "Innespeta", "Morampudi", "Aryapuram", "Danavai Peta", "Alcot Gardens",
                            "Syamalamba Temple Road", "Tilak Road", "Tadepalli"]

    system_message = f"""
    You are a helpful and knowledgeable restaurant recommendation assistant. Your role is to guide users in finding restaurants based on their preferences,
    such as cuisine, location, dietary restrictions. Be friendly, engaging, and concise while offering clear and accurate suggestions. Always prioritize user input and provide relevant follow-up questions if their request lacks details.

    Your final objective is to fill the values for the different keys ('Cuisine preferences', 'Dietary restrictions', 'Location') in the Python dictionary and be confident of the values.
    These key-value pairs define the user's profile. The Python dictionary looks like this:
    { {'Cuisine preferences': "values",
       'Dietary restrictions': "values",
       'Location': "values"} }

    All the values in the example dictionary are only representative values.\n
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalized:
    - To fill the cuisine preferences value, ask the user to choose one from {Cuisine_Preferences}.
    - To fill the Location value, ask the user to choose one from {Restaurant_Locations}.
    - To fill Dietary Restrictions value, first ask the user whether they have any dietary restrictions. If not, take it as 'No restrictions'. If yes, ask the user to select their dietary restriction from 'Vegan-friendly', 'Vegetarian-friendly', 'Gluten-free options'.
    - The values need to be inferred from the user's response.
    {delimiter}\n

    To fill the dictionary, you need to have the following chain of thoughts:
    Thought 1: Ask a question to understand the user's requirements.
    If their preferred cuisine preferences, dietary restrictions, or location is unclear, ask follow-up questions to understand their needs.
    You are trying to fill the values of all the keys {{ 'Cuisine Preferences', 'Dietary Restrictions', 'Location' }} in the Python dictionary by understanding the user's requirements.
    Identify the keys for which you can fill the values confidently using the understanding.
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step.
    Otherwise, rephrase the question to capture their profile clearly.\n

    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's requirements.
    If yes, move to the next Thought. If no, ask questions on the keys whose values you are unsure of.\n


    Thought 3: Check if you have correctly updated the values for the different keys in the Python dictionary.
    If you are not confident about any of the values, ask clarifying questions.\n

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, Show me Restaurants Nearby."
    Assistant: "Sure! please choose your nearest location from {Restaurant_Locations}".
    User: "Innespeta"
    Assistant: "Sure, There are some good restaurants nearby. Please choose your Cuisine Preferences from {Cuisine_Preferences}."
    User: "I primarily prefer South Indian."
    Assistant: "Good Choice!, Do you have any dietary restrictions?. Please Choose from {Dietary_Restrictions}."
    User: "I prefer Vegan friendly food."
    Assistant: "Thank you for providing that information. {example_user_dict}"\n
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, Show me some Vegan friendly Restaurants Nearby."
    Assistant: "Sure! Thanks for Specifying your dietary restrictions as Vegan-friendly .Now, Please choose your nearest location from {Restaurant_Locations}".{example_user_req_Diet}
    User: "Innespeta"
    Assistant: "Sure, There are some good restaurants nearby. Please choose your Cuisine Preferences from {Cuisine_Preferences}. {example_user_req_diet_loc}
    User: "I primarily prefer Vegan friendly food."
    Assistant: "Sure noted that, could you please provide your cuisine preference from {Cuisine_Preferences}"
    User: "sure, I prefer South Indian."
    Assistant: "Thank you for providing that information. {example_user_dict}"\n
    {delimiter}
    Start with a short welcome message and encourage the user to share their requirements.
    """

    return [{"role": "system", "content": system_message}]


In [98]:
# System message looks like this
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are a helpful and knowledgeable restaurant recommendation assistant. Your role is to guide users in finding restaurants based on their preferences,\n    such as cuisine, location, dietary restrictions. Be friendly, engaging, and concise while offering clear and accurate suggestions. Always prioritize user input and provide relevant follow-up questions if their request lacks details.\n\n    Your final objective is to fill the values for the different keys (\'Cuisine preferences\', \'Dietary restrictions\', \'Location\') in the Python dictionary and be confident of the values.\n    These key-value pairs define the user\'s profile. The Python dictionary looks like this:\n    {\'Cuisine preferences\': \'values\', \'Dietary restrictions\': \'values\', \'Location\': \'values\'}\n\n    All the values in the example dictionary are only representative values.\n\n    ####\n    Here are some instructions around the values for the different keys. If you do

In [99]:
# debug conversation has role and content lets look at its content
debug_conversation[0]['content']

'\n    You are a helpful and knowledgeable restaurant recommendation assistant. Your role is to guide users in finding restaurants based on their preferences,\n    such as cuisine, location, dietary restrictions. Be friendly, engaging, and concise while offering clear and accurate suggestions. Always prioritize user input and provide relevant follow-up questions if their request lacks details.\n\n    Your final objective is to fill the values for the different keys (\'Cuisine preferences\', \'Dietary restrictions\', \'Location\') in the Python dictionary and be confident of the values.\n    These key-value pairs define the user\'s profile. The Python dictionary looks like this:\n    {\'Cuisine preferences\': \'values\', \'Dietary restrictions\': \'values\', \'Location\': \'values\'}\n\n    All the values in the example dictionary are only representative values.\n\n    ####\n    Here are some instructions around the values for the different keys. If you do not follow this, you\'ll be he

In [100]:
# lets initialize conversation
system_message = initialize_conversation()
print(system_message[0]['content'])


    You are a helpful and knowledgeable restaurant recommendation assistant. Your role is to guide users in finding restaurants based on their preferences,
    such as cuisine, location, dietary restrictions. Be friendly, engaging, and concise while offering clear and accurate suggestions. Always prioritize user input and provide relevant follow-up questions if their request lacks details.

    Your final objective is to fill the values for the different keys ('Cuisine preferences', 'Dietary restrictions', 'Location') in the Python dictionary and be confident of the values.
    These key-value pairs define the user's profile. The Python dictionary looks like this:
    {'Cuisine preferences': 'values', 'Dietary restrictions': 'values', 'Location': 'values'}

    All the values in the example dictionary are only representative values.

    ####
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalized:
    - To fill t

In [101]:
# get_chat_completions function - this function perform LLM call to get LLM Response
# retry upto 6 attempts,waiting time min 1sec to maxing 20sec
 # Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

In [102]:
# testing openai functions defined above
input_prompt = 'What is the capital of france?'
messages = [{'role':'user', 'content':input_prompt}]
print(messages)

[{'role': 'user', 'content': 'What is the capital of france?'}]


In [103]:
get_chat_completions(messages)

'The capital of France is Paris.'

In [104]:
# get LLM response in JSon format
get_chat_completions(messages, json_format=True)

{'output': 'Paris'}

In [105]:
# creating a helper function which iterates any function any number of times, This is to check Consistency in LLM's response
def iterate_llm_response(funct, debug_response, num = 6):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter



In [106]:
# Example usage: test the consistency of responses from 'intent_confirmation_layer'
iterate_llm_response(get_chat_completions, messages)

Iteration: 0
{
    "output": "Paris"
}
--------------------------------------------------
Iteration: 1
{
    "output": "Paris"
}
--------------------------------------------------
Iteration: 2
{
    "output": "Paris"
}
--------------------------------------------------
Iteration: 3
{
    "output": "Paris"
}
--------------------------------------------------
Iteration: 4
{
    "output": "Paris"
}
--------------------------------------------------
Iteration: 5
{
    "output": "Paris"
}
--------------------------------------------------


In [107]:
# debugging to check assistant's response
debug_user_input = "Hi, show me nearby vegetarian restaurants"

In [108]:
debug_conversation.append({"role": "user", "content": debug_user_input})
print(debug_conversation[1]["content"]) # User Input
print(debug_conversation)

Hi, show me nearby vegetarian restaurants
[{'role': 'system', 'content': '\n    You are a helpful and knowledgeable restaurant recommendation assistant. Your role is to guide users in finding restaurants based on their preferences,\n    such as cuisine, location, dietary restrictions. Be friendly, engaging, and concise while offering clear and accurate suggestions. Always prioritize user input and provide relevant follow-up questions if their request lacks details.\n\n    Your final objective is to fill the values for the different keys (\'Cuisine preferences\', \'Dietary restrictions\', \'Location\') in the Python dictionary and be confident of the values.\n    These key-value pairs define the user\'s profile. The Python dictionary looks like this:\n    {\'Cuisine preferences\': \'values\', \'Dietary restrictions\': \'values\', \'Location\': \'values\'}\n\n    All the values in the example dictionary are only representative values.\n\n    ####\n    Here are some instructions around th

In [109]:
debug_response_assistant = get_chat_completions(debug_conversation)
display(debug_response_assistant)

'Sure! Do you have any specific location in mind for these vegetarian restaurants?'

In [110]:
# Let's append this to the conversation list
debug_conversation.append(({"role": "assistant", "content": debug_response_assistant}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are a helpful and knowledgeable restaurant recommendation assistant. Your role is to guide users in finding restaurants based on their preferences,\n    such as cuisine, location, dietary restrictions. Be friendly, engaging, and concise while offering clear and accurate suggestions. Always prioritize user input and provide relevant follow-up questions if their request lacks details.\n\n    Your final objective is to fill the values for the different keys (\'Cuisine preferences\', \'Dietary restrictions\', \'Location\') in the Python dictionary and be confident of the values.\n    These key-value pairs define the user\'s profile. The Python dictionary looks like this:\n    {\'Cuisine preferences\': \'values\', \'Dietary restrictions\': \'values\', \'Location\': \'values\'}\n\n    All the values in the example dictionary are only representative values.\n\n    ####\n    Here are some instructions around the values for the different keys. If you 

In [111]:
# function which checks moderation in user and assitant's response, To detect Unsafe content
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [112]:
# This must be flagged as it is unsafe
moderation_check('I want to self_harm')

'Flagged'

In [113]:
# lets moderation check debug_user_input
debug_moderation = moderation_check(debug_user_input)
debug_moderation

'Not Flagged'

In [114]:
# This function returns 'Yes' if all the specifications are gathered from the user. else returns 'No'
def intent_confirmation_layer(response_assistant):

    delimiter = "####"
    Dietary_Restrictions = ["No restrictions", "Vegan-friendly", "Vegetarian-friendly", "Gluten-free options"]
    Cuisine_Preferences = ["South Indian", "North Indian", "Chinese", "Continental", "Italian", "Andhra Cuisine"]
    Restaurant_Locations = ["Katheru", "Innespeta", "Morampudi", "Aryapuram", "Danavai Peta", "Alcot Gardens",
                            "Syamalamba Temple Road", "Tilak Road", "Tadepalli"]


    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 3 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    { {'Cuisine preferences': "values",
       'Dietary restrictions': "values",
       'Location': "values"} }
    The values of keys must match with respective Dietary_Restrictions from {Dietary_Restrictions}, Cuisine_Preferences from {Cuisine_Preferences} and Restaurant_locations from {Restaurant_Locations}
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    """

    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

In [115]:
debug_response_assistant

'Sure! Do you have any specific location in mind for these vegetarian restaurants?'

In [116]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)

{'result': 'No', 'reason': 'Missing keys: Cuisine preferences, Dietary restrictions, Location'}


In [117]:
# Example 1 - Let's check with the confirmation_layer if all the keys are present
debug_response_assistant_1 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{{'Cuisine preferences': "South Indian",
  'Dietary restrictions': "Vegan-friendly",
  'Location': "Innespeta"}}
"""
print(debug_response_assistant_1)


Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{'Cuisine preferences': "South Indian",
  'Dietary restrictions': "Vegan-friendly",
  'Location': "Innespeta"}



In [118]:
response = intent_confirmation_layer(debug_response_assistant_1)
print(response)

{'result': 'Yes'}


In [119]:
#Example 2
debug_response_assistant_1 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{{'Cuisine preference': "South Indian",
  'Dietary restrictions': "Vegan-friendly",
  'Location': '_'}}
"""
print(debug_response_assistant_1)


Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{'Cuisine preference': "South Indian",
  'Dietary restrictions': "Vegan-friendly",
  'Location': '_'}



In [120]:
response = intent_confirmation_layer(debug_response_assistant_1)
print(response)

{'result': 'No', 'reason': 'Location value is missing'}


In [121]:
# This Function converts the gathered user specifications to dictionary format
def dictionary_present(response):
    delimiter = "####"
    Dietary_Restrictions = ["No restrictions", "Vegan-friendly", "Vegetarian-friendly", "Gluten-free options"]
    Cuisine_Preferences = ["South Indian", "North Indian", "Chinese", "Continental", "Italian", "Andhra Cuisine"]
    Restaurant_Locations = ["Katheru", "Innespeta", "Morampudi", "Aryapuram", "Danavai Peta", "Alcot Gardens",
                            "Syamalamba Temple Road", "Tilak Road", "Tadepalli"]
    user_req = {'Cuisine preferences': "South Indian",
                'Dietary restrictions': "Vegan-friendly",
                'Location': "Innespeta"}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{'Cuisine preferences': "South Indian"/"North Indian"/"Chinese"/"Continental"/"Italian"/"Andhra Cuisine"  ,
                'Dietary restrictions': "No restrictions"/"Vegan-friendly"/"Vegetarian-friendly"/"Gluten-free options",
                'Location': "Katheru"/"Innespeta"/"Morampudi"/"Aryapuram"/"Danavai Peta"/"Alcot Gardens"/"Syamalamba Temple Road"/"Tilak Road"/"Tadepalli"
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: 'Cuisine preferences': "South Indian", 'Dietary restrictions': "Vegan-friendly", 'Location': "Innespeta"
            output 1: {{'Cuisine preferences': "South Indian", 'Dietary restrictions': "Vegan-friendly", 'Location': "Innespeta"}}

            input 2: {{'Cuisine preferences':     "South Indian", 'Dietary restrictions':     "Vegan-friendly", 'Location':     "Innespeta"}}
            output 2: {{'Cuisine preferences': "South Indian", 'Dietary restrictions': "Vegan-friendly", 'Location': "Innespeta"}}

            input 3: Here is your user profile 'Cuisine preferences': "North Indian", 'Dietary restrictions': "Gluten-free options", 'Location': "Morampudi"
            output 3: {{'Cuisine preferences': "North Indian", 'Dietary restrictions': "Gluten-free options", 'Location': "Morampudi"}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)
    return confirmation

In [122]:
debug_response_assistant_n = {'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}

In [123]:
response_dict_n = dictionary_present(debug_response_assistant_n)
display(response_dict_n)

{'Cuisine preferences': 'South Indian',
 'Dietary restrictions': 'Vegan-friendly',
 'Location': 'Innespeta'}

In [124]:
type(response_dict_n)

dict

In [125]:
debug_response_assistant_n = f"""Thank you for providing your requirements
Here are the restaurant specifications:
- 'Cuisine preference': 'South Indian'
- 'Dietary restrictions': 'Vegan-friendly'
- 'Location': 'Innespeta'

Please, Let me know if there's anything else I can assist you with!"""

In [126]:
response_dict_n = dictionary_present(debug_response_assistant_n)
display(response_dict_n)

{'Cuisine preferences': 'South Indian',
 'Dietary restrictions': 'Vegan-friendly',
 'Location': 'Innespeta'}

In [127]:
type(response_dict_n)

dict

In [128]:
# Check for LLM function's consistency
iterate_llm_response(dictionary_present, debug_response_assistant_n)

Iteration: 0
{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}
--------------------------------------------------
Iteration: 1
{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}
--------------------------------------------------
Iteration: 2
{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}
--------------------------------------------------
Iteration: 3
{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}
--------------------------------------------------
Iteration: 4
{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}
--------------------------------------------------
Iteration: 5
{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}
-------------------------

In [129]:
debug_conversation

[{'role': 'system',
  'content': '\n    You are a helpful and knowledgeable restaurant recommendation assistant. Your role is to guide users in finding restaurants based on their preferences,\n    such as cuisine, location, dietary restrictions. Be friendly, engaging, and concise while offering clear and accurate suggestions. Always prioritize user input and provide relevant follow-up questions if their request lacks details.\n\n    Your final objective is to fill the values for the different keys (\'Cuisine preferences\', \'Dietary restrictions\', \'Location\') in the Python dictionary and be confident of the values.\n    These key-value pairs define the user\'s profile. The Python dictionary looks like this:\n    {\'Cuisine preferences\': \'values\', \'Dietary restrictions\': \'values\', \'Location\': \'values\'}\n\n    All the values in the example dictionary are only representative values.\n\n    ####\n    Here are some instructions around the values for the different keys. If you 

In [130]:
debug_conversation = initialize_conversation() # load up the system message
debug_user_input = "Hi, Show me some Vegetarian Restaurants." # pass the user input
debug_moderation = moderation_check(debug_user_input) #check for moderation
if debug_moderation == "Not Flagged":
  debug_conversation.append({"role": "user", "content": debug_user_input}) #stored in the memory
  debug_response_assistant = get_chat_completions(debug_conversation)
  print(debug_response_assistant)
  debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
  debug_confirmation = intent_confirmation_layer(debug_response_assistant) #stored in the memory
  print(debug_confirmation['result'])
  if debug_confirmation['result']=="No":
    debug_user_input2 = "Yes, my dietary restriction is vegetarian friendly. I choose Innespeta" # pass the user input
    debug_moderation = moderation_check(debug_user_input2) #check for moderation
    if debug_moderation == "Not Flagged":
        debug_conversation.append({"role": "user", "content": debug_user_input2}) #stored in the memory
        debug_response_assistant = get_chat_completions(debug_conversation)
        print(debug_response_assistant)

Sure! Thanks for specifying your dietary restrictions as Vegetarian-friendly. Now, please choose your nearest location from ['Katheru', 'Innespeta', 'Morampudi', 'Aryapuram', 'Danavai Peta', 'Alcot Gardens', 'Syamalamba Temple Road', 'Tilak Road', 'Tadepalli']. {'Cuisine preferences': '_', 'Dietary restrictions': 'Vegetarian-friendly', 'Location': '_'}
No
Great choice! Innespeta is a nice location. Please choose your Cuisine Preferences from ['South Indian', 'North Indian', 'Chinese', 'Continental', 'Italian', 'Andhra Cuisine']. 


In [131]:
debug_conversation = initialize_conversation()
debug_user_input = "Hi, show me nearby restaurants."
debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_completions(debug_conversation)
debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
# After a series of conversation...
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}


In [132]:
print(response_dict_n == {'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'})
print(response_dict_n)

True
{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'}


# Stage 2

In [133]:
# This function maps each restaurant to its restaurant specifications in a dictionary format, just by raking user review as input
def restaurant_specs_map_layer(user_review):
    delimiter = "#####"
    Dietary_Restrictions = ["No restrictions", "Vegan-friendly", "Vegetarian-friendly", "Gluten-free options"]
    Cuisine_Preferences = ["South Indian", "North Indian", "Chinese", "Continental", "Italian", "Andhra Cuisine"]
    Restaurant_Locations = ["Katheru", "Innespeta", "Morampudi", "Aryapuram", "Danavai Peta", "Alcot Gardens",
                            "Syamalamba Temple Road", "Tilak Road", "Tadepalli"]

    restaurant_specs = {
        "Cuisine preferences":"(Cuisine of that restaurant)",
        "Dietary restrictions":"(Dietary restrictions of the restaurant)",
        "Location":"(restaurant location)"
    }

    prompt=f"""
    You are a Restaurant Specifications Classifier whose job is to extract the key specifications of restaurants and classify them as per their requirements.
    To analyze each restaurant specification, perform the following steps:
    Step 1: Extract the restaurant primary specifications from the user review {user_review}
    Step 2: Store the extracted specifications in {restaurant_specs} \
    Step 3: Classify each of the items in {restaurant_specs} based on the following rules: \
    {delimiter}
    Cuisine preferences: value must be one from {Cuisine_Preferences}
    Dietary restrictions: value mist be one from {Dietary_Restrictions}
    Location: value must be one from {Restaurant_Locations}

    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "I had a great time at Ravi's Kitchen in Aryapuram! They offer amazing Chinese dishes, perfect for anyone looking for no restrictions options."
    output 1: {{'Cuisine preferences': 'Chinese','Dietary restrictions':'No restrictions','Location':'Aryapuram'}}

    {delimiter}    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {restaurant_specs} for the following restaurant {user_review}."""
    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]

    response = get_chat_completions(messages, json_format = True)

    return response

In [134]:
restaurant_user_review_1 = "Sri Sai Akshaya is the go-to place in Innespeta for South Indian. They also cater to those needing vegan-friendly meals. Loved my experience!"
restaurant_specs_map_layer(restaurant_user_review_1)

{'Cuisine preferences': 'South Indian',
 'Dietary restrictions': 'Vegan-friendly',
 'Location': 'Innespeta'}

In [135]:
# Applying restaurant_specs_map_layer to the dataset
df['restaurant Dictionary'] = df['User Reviews'].apply(restaurant_specs_map_layer)
df

,Restaurant Name,Cuisine Preferences,Dietary Restrictions,Location,User Reviews,restaurant Dictionary
0,Sri Sai Akshaya,South Indian,Vegan-friendly,Innespeta,Sri Sai Akshaya is the go-to place in Innespet...,"{'Cuisine preferences': 'South Indian', 'Dieta..."
1,Swagath Grand,North Indian,Gluten-free options,Morampudi,Swagath Grand is the go-to place in Morampudi ...,"{'Cuisine preferences': 'North Indian', 'Dieta..."
2,Ravi's Kitchen,Chinese,No restrictions,Aryapuram,I had a great time at Ravi's Kitchen in Aryapu...,"{'Cuisine preferences': 'Chinese', 'Dietary re..."
3,Hotel River Bay,Continental,Vegan-friendly,Danavai Peta,Hotel River Bay in Danavai Peta is fantastic! ...,"{'Cuisine preferences': 'Continental', 'Dietar..."
4,Masala Darbar,Italian,Vegetarian-friendly,Alcot Gardens,"Visited Masala Darbar at Alcot Gardens, and I ...","{'Cuisine preferences': 'Italian', 'Dietary re..."
5,Spicy Village,Andhra Cuisine,No restrictions,Katheru,Craving Andhra Cuisine in Katheru? Check out S...,"{'Cuisine preferences': 'Andhra Cuisine', 'Die..."
6,Subbayya Gari Hotel,South Indian,No restrictions,Syamalamba Temple Road,Visited Subbayya Gari Hotel at Syamalamba Temp...,"{'Cuisine preferences': 'South Indian', 'Dieta..."
7,Sri Kanya Comforts,North Indian,Vegan-friendly,Tilak Road,Sri Kanya Comforts in Tilak Road is fantastic!...,"{'Cuisine preferences': 'North Indian', 'Dieta..."
8,Udupi Bhavan,Chinese,Vegetarian-friendly,Tadepalli,Udupi Bhavan in Tadepalli is fantastic! Their ...,"{'Cuisine preferences': 'Chinese', 'Dietary re..."
9,Biryani Zone,Continental,Gluten-free options,Katheru,Craving Continental in Katheru? Check out Biry...,"{'Cuisine preferences': 'Continental', 'Dietar..."


In [136]:
# to get all columns except restaurant dictionary column
df.loc[0][:-1]

,0
Restaurant Name,Sri Sai Akshaya
Cuisine Preferences,South Indian
Dietary Restrictions,Vegan-friendly
Location,Innespeta
User Reviews,Sri Sai Akshaya is the go-to place in Innespet...


In [137]:
df.columns

Index(['Restaurant Name', 'Cuisine Preferences', 'Dietary Restrictions', 'Location',
       'User Reviews', 'restaurant Dictionary'],
      dtype='object')

In [138]:
# comparing restaurant specifications with the user's specificaitons and returns all matched restaurants. If no restaurants matched it returns empty [] list
def compare_restaurant_specs_with_user(user_req_dict):
    selected_restaurants = []  # a list to store all matches
    for index, row in df.iterrows():
        if row['restaurant Dictionary'] == user_req_dict:
            selected_restaurants.append(dict(df.loc[index][:-1]))
    return selected_restaurants  # Return the list of all matches


In [139]:
# this returns the list of matched restaurants each list element contains [restaurant name + cuisine+ deitary specification+ location+ user review]matched
compare_restaurant_specs_with_user(response_dict_n)

[{'Restaurant Name': 'Sri Sai Akshaya',
  'Cuisine Preferences': 'South Indian',
  'Dietary Restrictions': 'Vegan-friendly',
  'Location': 'Innespeta',
  'User Reviews': 'Sri Sai Akshaya is the go-to place in Innespeta for South Indian. They also cater to those needing vegan-friendly meals. Loved my experience!'},
 {'Restaurant Name': 'Andhra Spice',
  'Cuisine Preferences': 'South Indian',
  'Dietary Restrictions': 'Vegan-friendly',
  'Location': 'Innespeta',
  'User Reviews': 'I had a great time at Andhra Spice in Innespeta! They offer amazing South Indian dishes, perfect for anyone looking for vegan-friendly options.'}]

In [140]:
# Matched Restaurants
top_restaurants = compare_restaurant_specs_with_user(response_dict_n)
top_restaurants

[{'Restaurant Name': 'Sri Sai Akshaya',
  'Cuisine Preferences': 'South Indian',
  'Dietary Restrictions': 'Vegan-friendly',
  'Location': 'Innespeta',
  'User Reviews': 'Sri Sai Akshaya is the go-to place in Innespeta for South Indian. They also cater to those needing vegan-friendly meals. Loved my experience!'},
 {'Restaurant Name': 'Andhra Spice',
  'Cuisine Preferences': 'South Indian',
  'Dietary Restrictions': 'Vegan-friendly',
  'Location': 'Innespeta',
  'User Reviews': 'I had a great time at Andhra Spice in Innespeta! They offer amazing South Indian dishes, perfect for anyone looking for vegan-friendly options.'}]

## Product Validation Layer

In [141]:
# This function will makes sure that recommended restaurants are not null.
def recommendation_validation(restaurant_recommendation):
  data = restaurant_recommendation
  data1 = []
  if len(data) != 0:
    for i in range(len(data)):
      data1.append(data[i])
  return data1

In [142]:
# Applying above function
validated_data = recommendation_validation(top_restaurants)
display(validated_data,'\n')

[{'Restaurant Name': 'Sri Sai Akshaya',
  'Cuisine Preferences': 'South Indian',
  'Dietary Restrictions': 'Vegan-friendly',
  'Location': 'Innespeta',
  'User Reviews': 'Sri Sai Akshaya is the go-to place in Innespeta for South Indian. They also cater to those needing vegan-friendly meals. Loved my experience!'},
 {'Restaurant Name': 'Andhra Spice',
  'Cuisine Preferences': 'South Indian',
  'Dietary Restrictions': 'Vegan-friendly',
  'Location': 'Innespeta',
  'User Reviews': 'I had a great time at Andhra Spice in Innespeta! They offer amazing South Indian dishes, perfect for anyone looking for vegan-friendly options.'}]

'\n'

# Stage 3

In [143]:
# This function sets the system messgae to recommend matched restaurants in short. Without showing dictionaries
def initialize_conv_reco(restaurants):
    system_message = f"""
    You are an intelligent restaurant recomendation expert and you are tasked with the objective to \
    solve the user queries about any restaurant specifications like cuisine preference, dietary specifications, \
    location from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each restaurant in the following format,:
    1. <Restaurant Name> : <Major specifications of the Restaurant>
    2. <Restaurant Name> : <Major specifications of the Restaurant>

    """
    user_message = f""" These are the user's products: {restaurants}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
    # conversation_final = conversation[0]['content']
    return conversation

In [144]:
debug_conversation_reco = initialize_conv_reco(top_restaurants)
debug_conversation_reco

[{'role': 'system',
  'content': '\n    You are an intelligent restaurant recomendation expert and you are tasked with the objective to     solve the user queries about any restaurant specifications like cuisine preference, dietary specifications,     location from the catalogue in the user message     You should keep the user profile in mind while answering the questions.\n    Start with a brief summary of each restaurant in the following format,:\n    1. <Restaurant Name> : <Major specifications of the Restaurant>\n    2. <Restaurant Name> : <Major specifications of the Restaurant>\n\n    '},
 {'role': 'user',
  'content': " These are the user's products: [{'Restaurant Name': 'Sri Sai Akshaya', 'Cuisine Preferences': 'South Indian', 'Dietary Restrictions': 'Vegan-friendly', 'Location': 'Innespeta', 'User Reviews': 'Sri Sai Akshaya is the go-to place in Innespeta for South Indian. They also cater to those needing vegan-friendly meals. Loved my experience!'}, {'Restaurant Name': 'Andhr

In [145]:
# Assistant's response
debug_recommendation = get_chat_completions(debug_conversation_reco)
print(debug_recommendation + '\n')

1. Sri Sai Akshaya : A South Indian restaurant located in Innespeta that offers vegan-friendly meals. Users have reviewed it as the go-to place for South Indian cuisine in the area with positive remarks about the vegan-friendly options.

2. Andhra Spice : Another South Indian restaurant in Innespeta known for its vegan-friendly dishes. Customers have complimented the restaurant for its great South Indian food and variety of vegan options available.



# Combining all 3 stages

In [146]:
# combination of all functions. Running all functions in order to chat and expect response from an LLM until 'exit' appears.
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction + '\n')

    restaurants = None

    user_input = ''

    while(user_input != "exit"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if restaurants is None:

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completions(conversation)
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break


            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:",confirmation.get('result'))

            if "No" in confirmation.get('result'):
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                print("\n" + str(response_assistant) + "\n")
                print('\n' + "Restaurant Specifications extracted!" + '\n')

                response = dictionary_present(response_assistant)

                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                restaurants = compare_restaurant_specs_with_user(response)

                print("Matched Restaurants are:", restaurants)
                validated_reco = recommendation_validation(restaurants)

                conversation_reco = initialize_conv_reco(validated_reco)

                conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})

                recommendation = get_chat_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    display("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})

                print(str(recommendation) + '\n')
        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

In [147]:
# by running this cell. we are initializing chatbot to run.
dialogue_mgmt_system()

"Hello! Welcome to the restaurant recommendation assistant. I'm here to help you find the perfect restaurant based on your preferences. Please feel free to share your requirements, such as cuisine preferences, dietary restrictions, and location. Let's get started!\n"

Hi, Show me restaurants near to innespeta which have south indian cuisine with vegan friendly diet 
Intent Confirmation Yes/No: No

Sure! Thank you for specifying your requirements. 
To assist you better, could you please confirm your primary choice for South Indian cuisine and if you have any other dietary restrictions apart from Vegan-friendly?

no
Intent Confirmation Yes/No: Yes

Great! Just to confirm, your Cuisine Preferences are South Indian, Dietary Restrictions are Vegan-friendly, and the Location is Innespeta. 

Your profile is:
{'Cuisine preferences': 'South Indian', 'Dietary restrictions': 'Vegan-friendly', 'Location': 'Innespeta'} 

Is there anything else you would like to add or inquire about?


Restaurant Specifications extracted!

Thank you for providing all the information. Kindly wait, while I fetch the products: 

Matched Restaurants are: [{'Restaurant Name': 'Sri Sai Akshaya', 'Cuisine Preferences': 'South Indian', 'Dietary Restrictions': 'Vegan-friendly', 'Location'

In [148]:
df


,Restaurant Name,Cuisine Preferences,Dietary Restrictions,Location,User Reviews,restaurant Dictionary
0,Sri Sai Akshaya,South Indian,Vegan-friendly,Innespeta,Sri Sai Akshaya is the go-to place in Innespet...,"{'Cuisine preferences': 'South Indian', 'Dieta..."
1,Swagath Grand,North Indian,Gluten-free options,Morampudi,Swagath Grand is the go-to place in Morampudi ...,"{'Cuisine preferences': 'North Indian', 'Dieta..."
2,Ravi's Kitchen,Chinese,No restrictions,Aryapuram,I had a great time at Ravi's Kitchen in Aryapu...,"{'Cuisine preferences': 'Chinese', 'Dietary re..."
3,Hotel River Bay,Continental,Vegan-friendly,Danavai Peta,Hotel River Bay in Danavai Peta is fantastic! ...,"{'Cuisine preferences': 'Continental', 'Dietar..."
4,Masala Darbar,Italian,Vegetarian-friendly,Alcot Gardens,"Visited Masala Darbar at Alcot Gardens, and I ...","{'Cuisine preferences': 'Italian', 'Dietary re..."
5,Spicy Village,Andhra Cuisine,No restrictions,Katheru,Craving Andhra Cuisine in Katheru? Check out S...,"{'Cuisine preferences': 'Andhra Cuisine', 'Die..."
6,Subbayya Gari Hotel,South Indian,No restrictions,Syamalamba Temple Road,Visited Subbayya Gari Hotel at Syamalamba Temp...,"{'Cuisine preferences': 'South Indian', 'Dieta..."
7,Sri Kanya Comforts,North Indian,Vegan-friendly,Tilak Road,Sri Kanya Comforts in Tilak Road is fantastic!...,"{'Cuisine preferences': 'North Indian', 'Dieta..."
8,Udupi Bhavan,Chinese,Vegetarian-friendly,Tadepalli,Udupi Bhavan in Tadepalli is fantastic! Their ...,"{'Cuisine preferences': 'Chinese', 'Dietary re..."
9,Biryani Zone,Continental,Gluten-free options,Katheru,Craving Continental in Katheru? Check out Biry...,"{'Cuisine preferences': 'Continental', 'Dietar..."


In [150]:
dialogue_mgmt_system()


'Hello! Welcome to the restaurant recommendation assistant. I can help you find the perfect restaurant based on your preferences. Please go ahead and let me know your requirements, such as cuisine preferences, dietary restrictions, and location. Feel free to share any details to help me assist you better.\n'

show me chinese restaurants with out diet restrictions
Intent Confirmation Yes/No: No

Hello! I can help you with that. Please choose your nearest location from ['Katheru', 'Innespeta', 'Morampudi', 'Aryapuram', 'Danavai Peta', 'Alcot Gardens', 'Syamalamba Temple Road', 'Tilak Road', 'Tadepalli'].

aryprm
Intent Confirmation Yes/No: No

Great choice! Now, please choose your Cuisine Preferences from ['South Indian', 'North Indian', 'Chinese', 'Continental', 'Italian', 'Andhra Cuisine'].

chinese
Intent Confirmation Yes/No: No

Perfect! Do you have any dietary restrictions such as 'Vegan-friendly', 'Vegetarian-friendly', 'Gluten-free options', or 'No restrictions'?

no
Intent Confirmation Yes/No: Yes

Thank you for the information. Based on your preferences, the values for the keys are as follows: 
- 'Cuisine preferences': 'Chinese'
- 'Dietary restrictions': 'No restrictions'
- 'Location': 'Aryapuram'


Restaurant Specifications extracted!

Thank you for providing all the information. Ki

In [151]:
dialogue_mgmt_system()


'Hello! Welcome to the restaurant recommendation assistant. I can help you find the perfect restaurant based on your preferences. Please go ahead and let me know your requirements.\n'

show me restaurants near morampudi
Intent Confirmation Yes/No: No

Sure! Please choose your Cuisine Preferences from ['South Indian', 'North Indian', 'Chinese', 'Continental', 'Italian', 'Andhra Cuisine'].

no idea
Intent Confirmation Yes/No: No

No worries! Do you have any dietary restrictions like Vegan-friendly, Vegetarian-friendly, or Gluten-free options?

no specifications
Intent Confirmation Yes/No: No

Understood. Please choose your nearest location from ['Katheru', 'Innespeta', 'Morampudi', 'Aryapuram', 'Danavai Peta', 'Alcot Gardens', 'Syamalamba Temple Road', 'Tilak Road', 'Tadepalli'].

morampudi
Intent Confirmation Yes/No: No

Great choice! Based on your location in Morampudi, here are some good restaurants nearby. Please choose your Cuisine Preferences from ['South Indian', 'North Indian', 'Chinese', 'Continental', 'Italian', 'Andhra Cuisine'].

ok south indian
Intent Confirmation Yes/No: No

Perfect! Do you have any dietary restrictions like Vegan-friendly, Vegetarian-fri